## Download latest Jars

In [2]:
dbutils.fs.mkdirs("dbfs:/FileStore/jars/")

In [3]:
%sh
cd ../../dbfs/FileStore/jars/
sudo wget -O cudf-22.02.0-cuda11.jar https://repo1.maven.org/maven2/ai/rapids/cudf/22.02.0/cudf-22.02.0-cuda11.jar
sudo wget -O rapids-4-spark_2.12-22.02.0.jar https://repo1.maven.org/maven2/com/nvidia/rapids-4-spark_2.12/22.02.0/rapids-4-spark_2.12-22.02.0.jar
sudo wget -O xgboost4j_3.0-1.4.2-0.2.0.jar https://repo1.maven.org/maven2/com/nvidia/xgboost4j_3.0/1.4.2-0.2.0/xgboost4j_3.0-1.4.2-0.2.0.jar
sudo wget -O xgboost4j-spark_3.0-1.4.2-0.2.0.jar https://repo1.maven.org/maven2/com/nvidia/xgboost4j-spark_3.0/1.4.2-0.2.0/xgboost4j-spark_3.0-1.4.2-0.2.0.jar
ls -ltr

# Your Jars are downloaded in dbfs:/FileStore/jars directory

### Create a Directory for your init script

In [5]:
dbutils.fs.mkdirs("dbfs:/databricks/init_scripts/")

In [6]:
dbutils.fs.put("/databricks/init_scripts/init.sh","""
#!/bin/bash
sudo cp /dbfs/FileStore/jars/xgboost4j_3.0-1.4.2-0.2.0.jar /databricks/jars/spark--maven-trees--ml--7.x--xgboost--ml.dmlc--xgboost4j_2.12--ml.dmlc__xgboost4j_2.12__1.0.0.jar
sudo cp /dbfs/FileStore/jars/cudf-22.02.0-cuda11.jar /databricks/jars/
sudo cp /dbfs/FileStore/jars/rapids-4-spark_2.12-22.02.0.jar /databricks/jars/
sudo cp /dbfs/FileStore/jars/xgboost4j-spark_3.0-1.4.2-0.2.0.jar /databricks/jars/spark--maven-trees--ml--7.x--xgboost--ml.dmlc--xgboost4j-spark_2.12--ml.dmlc__xgboost4j-spark_2.12__1.0.0.jar
sudo wget -O /etc/apt/preferences.d/cuda-repository-pin-600 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-ubuntu1804.pin
sudo wget -O ~/cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb https://developer.download.nvidia.com/compute/cuda/11.0.3/local_installers/cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
sudo dpkg -i ~/cuda-repo-ubuntu1804-11-0-local_11.0.3-450.51.06-1_amd64.deb
sudo apt-key add /var/cuda-repo-ubuntu1804-11-0-local/7fa2af80.pub
sudo apt-get update
sudo apt -y install cuda-toolkit-11-0
""", True)

### Confirm your init script is in the new directory

In [8]:
%sh
cd ../../dbfs/databricks/init_scripts
pwd
ls -ltr

### Download the Mortgage Dataset into your local machine and upload Data using import Data

In [10]:
dbutils.fs.mkdirs("dbfs:/FileStore/tables/")

In [11]:
%sh
cd /dbfs/FileStore/tables/
wget -O mortgage.zip https://rapidsai-data.s3.us-east-2.amazonaws.com/spark/mortgage.zip
ls
unzip mortgage.zip

In [12]:
%sh
pwd
cd ../../dbfs/FileStore/tables
ls -ltr mortgage/csv/*

### Delete the duplicate xgboost4j jars

In [ ]:
rm -f /databricks/jars/spark--maven-trees--ml--7.x--xgboost--ml.dmlc--xgboost4j_2.12--ml.dmlc__xgboost4j_2.12__1.0.0.jar
rm -f /databricks/jars/spark--maven-trees--ml--7.x--xgboost--ml.dmlc--xgboost4j-spark_2.12--ml.dmlc__xgboost4j-spark_2.12__1.0.0.jar

### Next steps

1. Edit your cluster, adding an initialization script from `dbfs:/databricks/init_scripts/init.sh` in the "Advanced Options" under "Init Scripts" tab
2. Reboot the cluster
3. Go to "Libraries" tab under your cluster and install `dbfs:/FileStore/jars/xgboost4j-spark_3.0-1.4.2-0.2.0.jar` in your cluster by selecting the "DBFS" option for installing jars
4. Import the mortgage example notebook from `https://github.com/NVIDIA/spark-rapids-examples/blob/branch-22.02/examples/Spark-ETL+XGBoost/mortgage/notebooks/python/mortgage-gpu.ipynb`
5. Inside the mortgage example notebook, update the data paths
  `train_data = reader.schema(schema).option('header', True).csv('/data/mortgage/csv/small-train.csv')`
  `trans_data = reader.schema(schema).option('header', True).csv('/data/mortgage/csv/small-trans.csv')`